# 📋 Trabalho - Case de negócios com BSP

**Prof.:** Thiago Bluhm<br>
**Disc.:** Data Harvesting<br>
**Turma:** Ciência de Dados - T9

**Alunos:**
- Caio da Silva Nascimento - Mat.: 2428455
- Rommel Assunção de Oliveira Sousa - Mat.: 2427990

**Objetivo:** A partir do conhecimento adiquirido em aula, criamos uma processo de raspagem de dados com o objetivo de coletar valores de automóveis novos e usados, para utilizar como objeto de apoio e tomada de deciões na precificação em uma consercionária.

O site utilizado para a busca foi o Web Motors, referência em site de compras de automoveis novos e usados.

#### 📲 Importando bibliotecas

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

### 🌐 Referenciando site (Web Motors)

**Obs.:** Como não foi possível realizar a requisição direta, foi necessário baixar o HTML do site para realizar a raspagem.

In [ ]:
# Carregando o HTML
html = urlopen('file:webmotors.html')

# Parseando o HTML com o BeautifulSoup
bs = BeautifulSoup(html, 'html.parser')

#### 🔍 Realizando análise exploratória

In [14]:
bs.find('div', class_='layout-main-content_Search__ax1VB')

<div class="layout-main-content_Search__ax1VB"><div id="search-container"><div class="search-sort-selector_Container___JD3G"><p class="_body-regular-small_qtpsh_152 d-desktop-block" data-qa="research_container">3.546 anúncios encontrados</p><p class="_body-regular-small_qtpsh_152 d-mobile-block" data-qa="research_container">3.546 anúncios</p><div class="_Container_1cp76_1 _light_1cp76_43"><div class="_Svg_1cp76_28"><svg class="_Svg_13gxs_1" height="24" viewbox="0 0 32 32" width="24"><path class="_Path_13gxs_5" d="M11.7068 8.293L7.70676 4.293C7.61476 4.201 7.50376 4.128 7.38176 4.077C7.13776 3.976 6.86176 3.976 6.61776 4.077C6.49576 4.128 6.38476 4.201 6.29276 4.293L2.29276 8.293C1.90176 8.684 1.90176 9.316 2.29276 9.707C2.68376 10.098 3.31576 10.098 3.70676 9.707L5.99976 7.414V19C5.99976 19.552 6.44776 20 6.99976 20C7.55176 20 7.99976 19.552 7.99976 19V7.414L10.2928 9.707C10.4878 9.902 10.7438 10 10.9998 10C11.2558 10 11.5118 9.902 11.7068 9.707C12.0978 9.316 12.0978 8.684 11.7068 8.29

In [13]:
bs.find('div', class_='_Container_70j0p_1').find('div', class_='_Body_1jfph_1').get_text(strip='', separator='|')

'PORSCHE CAYENNE|3.0 v6 e-hybrid awd tiptronic s|2021/2022|7.237 Km|Fortaleza (CE)|R$\xa0639.000|Ver parcelas'

#### ⚙️ Raspagem das informações alvo

In [15]:
# Definindo o dataframe para armazenar os dados
df = pd.DataFrame(columns=['Nome', 'Versão', 'Ano', 'Quilometragem', 'Localização', 'Preço', 'Oferta'])

In [16]:
# Armazenando as informações dos carros
carros = bs.find_all('div', class_='_Container_70j0p_1')

# Adicionando as informações dos carros no dataframe
for carro in carros:
    descricao = carro.find('div', class_='_Body_1jfph_1').get_text(strip='', separator='|')

    if 'Oferta Destaque' == descricao.split('|')[0]:
        nome_carro = descricao.split('|')[1]
        versao_carro = descricao.split('|')[2]
        ano_carro = descricao.split('|')[3]
        km_carro = descricao.split('|')[4]
        local_carro = descricao.split('|')[5]
        preco_carro = descricao.split('|')[6]
        oferta_carro = 'Sim'
    else:
        nome_carro = descricao.split('|')[0]
        versao_carro = descricao.split('|')[1]
        ano_carro = descricao.split('|')[2]
        km_carro = descricao.split('|')[3]
        local_carro = descricao.split('|')[4]
        preco_carro = descricao.split('|')[5]
        oferta_carro = 'Não'

    df = pd.concat([df, pd.DataFrame({'Nome': [nome_carro],
                                     'Versão': [versao_carro],
                                     'Ano': [ano_carro],
                                     'Quilometragem': [km_carro],
                                     'Localização': [local_carro],
                                     'Preço': [preco_carro],
                                     'Oferta': [oferta_carro]})], ignore_index=True)

    # print(f'Descrição: {descricao}')
    # print(f'Nome do carro: {nome_carro}')
    # print(f'Versão do carro: {versao_carro}')
    # print(f'Ano do carro: {ano_carro}')
    # print(f'Quilometragem do carro: {km_carro}')
    # print(f'Localização do carro: {local_carro}')
    # print(f'Preço do carro: {preco_carro}')
    # print(f'Oferta destaque: {oferta_carro}')
    # print('---')

# Exibindo o dataframe
df

,Nome,Versão,Ano,Quilometragem,Localização,Preço,Oferta
0,PORSCHE CAYENNE,3.0 v6 e-hybrid awd tiptronic s,2021/2022,7.237 Km,Fortaleza (CE),R$ 639.000,Não
1,AUDI A5,2.0 tfsi mhev sportback s line s tronic,2023/2024,10.000 Km,Fortaleza (CE),R$ 299.990,Não
2,CHEVROLET ONIX,1.0 turbo flex ltz automático,2023/2024,13.260 Km,Sobral (CE),R$ 96.390,Sim
3,JEEP RENEGADE,1.3 t270 turbo flex longitude at6,2023/2024,26.093 Km,Juazeiro do Norte (CE),R$ 120.690,Sim
4,CHEVROLET TRACKER,1.0 turbo flex ltz automático,2021/2022,46.730 Km,Fortaleza (CE),R$ 104.900,Não
5,AUDI A3,2.0 tfsi gasolina sedan performance black s-tr...,2025/2025,0 Km,Fortaleza (CE),R$ 349.990,Não
6,CITROËN C3 AIRCROSS,1.0 turbo 200 flex feel pack 7l cvt,2024/2025,18.926 Km,Juazeiro do Norte (CE),R$ 117.690,Sim
7,FIAT FIORINO,1.4 mpi furgão endurance 8v flex 2p manual,2023/2024,29.679 Km,Sobral (CE),R$ 95.590,Sim
8,PORSCHE CAYENNE,3.0 v6 s e-hybrid coupé awd tiptronic s,2024/2025,3.469 Km,Fortaleza (CE),R$ 899.000,Não
9,PORSCHE CAYENNE,3.0 v6 e-hybrid coupé awd tiptronic s,2024/2024,2.010 Km,Fortaleza (CE),R$ 800.000,Não


In [17]:
# Exportando o dataframe para um arquivo CSV
df.to_csv('carros.csv', index=False)